In [ ]:
import os
import json
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from tqdm import tqdm
from peft import LoraConfig


import torch
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
Gemma2ForCausalLM
)
from trl import SFTTrainer
from datasets import load_dataset, Dataset, DatasetDict
from accelerate import Accelerator
import peft

In [ ]:
# #QLoRA
# lora_config = LoraConfig(
#     r=6,#멀티헤드어텐션 헤드 개수
#     lora_alpha = 8, #어텐션 계수 스케일
#     lora_dropout = 0.05, #드롭아웃 비율
#     target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
#     task_type="CAUSAL_LM",
# )

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16
# )

In [ ]:
model_id = "google/gemma-2-2b-it"

os.environ["HF_TOKEN"] = ""

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map=None,
    trust_remote_code=True,
    token=os.environ["HF_TOKEN"]
)



# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True
)
tokenizer.use_default_system_prompt = False

In [ ]:
file_path = 'train.csv'
train_data = pd.read_csv(file_path)

train_data = train_data.sample(frac=1).reset_index(drop=True)

val_data=train_data[:50]
train_data = train_data[50:]

val_label_df = val_data[['question', 'answer']]

train_data["prompt"] = (
    "<start_of_turn>user\n"
    "당신은 뉴스 기사를 바탕으로 질문에 대해 간결하고 정확하게 답변해주는 어시스턴트입니다.\n"
    "다음에 주어지는 기사 내용을 잘 읽고, 그에 대한 질문에 단답형으로 응답해주세요.\n\n"
    "기사 내용:\n" + train_data['context'] + "\n\n"
    "질문:\n" + train_data['question'] + "\n"
    "질문에 대한 답변은 핵심만 포함된 1~2단어 수준의 짧은 응답이면 됩니다.\n"
    "<end_of_turn>\n"
    "<start_of_turn>model\n"
    "Answer: " + train_data['answer'] + "\n"
    "<end_of_turn>"
)

In [ ]:
data = Dataset.from_pandas(train_data)

In [ ]:
data = data.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

In [ ]:
def formatting_func(example):
    return example['prompt']


trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    max_seq_length=None,
    args=TrainingArguments(
        output_dir="outputs6",
        num_train_epochs = 2,
        max_steps=-1,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=2,
        learning_rate=5e-4,
        fp16=True,
        logging_steps=1000,
        push_to_hub=False,
        report_to='none',
        save_strategy='steps',
        save_steps=100,
        
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,

)
trainer.train()

In [ ]:
ADAPTER_MODEL = "lora_adapter"
trainer.model.save_pretrained(ADAPTER_MODEL)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.float16)

In [ ]:
from peft import PeftModel


In [ ]:
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

In [ ]:
model = model.merge_and_unload()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_path = "./model_3"

# 모델과 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id,use_auth_token=os.environ['HF_TOKEN'])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [ ]:
val_data["prompt"] = (
    "<start_of_turn>user\n"
    + "주어진 Context를 토대로 Question에 답변해\n"  
    + "Context: " + val_data['context'] + "\n"
    + "Question: " + val_data['question'] + "\n"
    + "<end_of_turn>\n<start_of_turn>model\n"
    + "Answer: " + val_data['answer'] + "\n"
    + "<end_of_turn>"
)

In [ ]:
val_data['prompt'][0]

In [ ]:
val_data

In [ ]:
response = qa_pipeline(question_prompt, max_new_tokens=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)[0]['generated_text']

In [ ]:
response

In [ ]:
def normalize_answer(s):
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text)
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)
        return text

    def white_space_fix(text):
        '''연속된 공백일 경우 하나의 공백으로 대체'''
        return ' '.join(text.split())

    def remove_punc(text):
        '''구두점 제거'''
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        '''소문자 전환'''
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()

    # 문자 단위로 f1-score를 계산 합니다.
    prediction_Char = []
    for tok in prediction_tokens:
        now = [a for a in tok]
        prediction_Char.extend(now)

    ground_truth_Char = []
    for tok in ground_truth_tokens:
        now = [a for a in tok]
        ground_truth_Char.extend(now)

    common = Counter(prediction_Char) & Counter(ground_truth_Char)
    num_same = sum(common.values())
    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(prediction_Char)
    recall = 1.0 * num_same / len(ground_truth_Char)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

def evaluate(ground_truth_df, predictions_df):
    predictions = dict(zip(predictions_df['question'], predictions_df['answer']))
    f1 = exact_match = total = 0

    for index, row in ground_truth_df.iterrows():
        question_text = row['question']
        ground_truths = row['answer']
        total += 1
        if question_text not in predictions:
            continue
        prediction = predictions[question_text]
        f1 = f1 + f1_score(prediction, ground_truths)

    f1 = 100.0 * f1 / total
    return {'f1': f1}

In [ ]:
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
# 모델로 추론 후, 전처리를 수행한 뒤, 완성된 정답으로 반환합니다.
def generate_response(question_prompt):
    # 생성할 최대 토큰 수와, 답변 생성 수, 패딩 토큰의 idx를 지정하여 모델 파이프 라인을 설정하고, 답변을 생성합니다.
    response = qa_pipeline(question_prompt, max_new_tokens=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)[0]['generated_text']
    if "Answer:" in response:
            # Answer: 이후에 생성된 토큰 들만을 답변으로 사용합니다.
            response = response.split("Answer:", 1)[1][:20]

            # 토큰 반복 생성 및 노이즈 토큰 관련 처리
            if "Que" in response:
                response = response.split("Que", 1)[0]
            if "⊙" in response:
                response = response.split("⊙", 1)[0]
            if "Con" in response:
                response = response.split("Con", 1)[0]
    return response

In [ ]:
predict_dict = {}
count = 0

for index, row in val_data.iterrows():
    try:
        context = row['context']
        question = row['question']


        if context is not None and question is not None:
            # question_prompt = f"너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. \
            #                     Question에 대한 답변만 있는 한 단어로 최대한 간결하게 답변하도록 해. \
            #                     # Context: {context} Question: {question}\n Answer:"
            question_prompt =f"당신은 주어진 Context를 기반으로 Question에 답하는 챗봇입니다.\
답변을 할 때, 반드시 context를 참고하세요.\
 문장이 아닌 간결한 단답이어야 합니다.\
Context: {context} Question: {question}\n Answer:"
#             question_prompt=f"당신은 주어진 Context를 기반으로 Question에 답하는 챗봇입니다.\
# 답변을 할 때, 반드시 context를 참고하세요.\
# 문장이 아닌 간결한 단답이어야 합니다.\
# 이 답변은 저의 대학 입시에 매우 중요합니다. 저를 위해 꼭 정답을 찾아주세요.\
# Context: {context} Question: {question}\n Answer:"
            # question_prompt = f"Context를 바탕으로 Question에 단답형으로 답변해 주세요.\
            #                     답변은 꼭 Context에 있는 내용이어야 해. \
            #                      Context: {context} Question: {question}\n Answer:"
            # question_prompt = f"주어진 Context를 기반으로 Question에 대해 간결하게, 한 단어로 답변하는 챗봇입니다. \
            #                      Context: {context} Question: {question}\n Answer:"
    
            answer = generate_response(question_prompt)
            predict_dict[question] = answer
        else:
            predict_dict[question] = 'Invalid question or context'

        print("Answer for question:", question, ":", predict_dict[question])
        count += 1
        print("Processed count:", count)
    except Exception as e:
        print(f"Error processing question {e}")
val_inf_df = pd.DataFrame(list(predict_dict.items()), columns=['question', 'answer'])
val_inf_df.head()
# f1-score 를 출력합니다.
results = evaluate(val_inf_df, val_label_df)
print(results)

In [ ]:
file_path = 'prj_data/test.csv'
test_data = pd.read_csv(file_path)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

submission_dict = {}

# 문서 쪼개기 및 벡터화 - 전체 test context에 대해 사전 구축
docs = text_splitter.create_documents(test_data['context'].dropna().tolist())
vectordb = FAISS.from_documents(docs, embedding_model)

# QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=False
)

# 추론 루프
for index, row in test_data.iterrows():
    try:
        context = row['context']
        question = row['question']
        qid = row['id']

        if context is not None and question is not None:
            # 프롬프트 구성
            question_prompt = (
                "<start_of_turn>user\n"
                "당신은 뉴스 기사를 바탕으로 질문에 대해 간결하고 정확하게 답변해주는 어시스턴트입니다.\n"
                "다음에 주어지는 기사 내용을 잘 읽고, 그에 대한 질문에 단답형으로 응답해주세요.\n\n"
                "기사 내용:\n" + context + "\n\n"
                "질문:\n" + question + "\n"
                "질문에 대한 답변은 핵심만 포함된 1~2단어 수준의 짧은 응답이면 됩니다.\n"
                "<end_of_turn>\n"
                "<start_of_turn>model\n"
            )

            # RAG를 활용한 답변 생성
            response = qa_chain({"query": question_prompt})
            answer = response["result"].strip()
            submission_dict[qid] = answer
        else:
            submission_dict[qid] = 'Invalid question or context'

    except Exception as e:
        print(f"Error processing index {index}, id={qid}: {e}")
        submission_dict[qid] = 'Error'


In [ ]:
df = pd.DataFrame(list(submission_dict.items()), columns=['id', 'answer'])
df.to_csv( './submission2.csv', index=False, encoding='utf-8-sig')

In [ ]:
n1=val_data['context'][0]
n2=val_data['question'][0]

In [ ]:
messages = [
    {
        "role": "user",
        "content": "{}를 토대로 {}에 답변해주세요.:\n\n{}".format(n1, n2)
    }
]

In [ ]:
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
response = qa_pipeline(question_prompt, max_new_tokens=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)[0]['generated_text']

In [ ]:
response

In [ ]:
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

In [ ]:
df = pd.DataFrame(list(submission_dict.items()), columns=['id', 'answer'])
df['answer'] = df['answer'].apply(lambda x: re.sub(r'\n', '', x))
df.to_csv( './submission.csv', index=False, encoding='utf-8-sig')